# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv")
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Esperance,-33.8667,121.9000,69.80,43,2,12.66,AU,1610342210
1,1,Thompson,55.7435,-97.8558,8.60,85,75,2.30,CA,1610342110
2,2,Punta Arenas,-53.1500,-70.9167,39.20,70,0,14.97,CL,1610342165
3,3,Saint Anthony,45.0205,-93.2180,26.60,80,90,6.91,US,1610342212
4,4,Port Alfred,-33.5906,26.8910,69.01,85,96,1.99,ZA,1610342213
...,...,...,...,...,...,...,...,...,...,...
589,1283,Saskylakh,71.9167,114.0833,-8.61,87,100,15.41,RU,1610342970
590,1284,Mount Gambier,-37.8333,140.7667,95.00,14,100,27.63,AU,1610342971
591,1285,Cefalù,38.0386,14.0229,55.00,81,89,1.01,IT,1610342971
592,1287,Lambaréné,-0.7001,10.2406,74.07,95,77,2.39,GA,1610342973


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

humidity = df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
heat_layer.max_intensity=100
heat_layer.point_radius=4

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = df.loc[(df["Max Temp"]>= 68) & (df["Max Temp"]<= 77) & (df["Wind Speed"] < 8) & (df["Cloudiness"] < 1), :]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,61,San Rafael,-34.6177,-68.3301,69.55,65,0,2.46,AR,1610342232
167,269,Gisborne,-38.6533,178.0042,72.00,73,0,4.00,NZ,1610342378
339,718,Paraíso,9.8383,-83.8656,69.80,73,0,6.91,CR,1610342640
429,968,Wamba,8.9333,8.6000,68.11,25,0,1.90,NG,1610342785
459,1064,Puntarenas,9.9763,-84.8384,69.80,73,0,6.91,CR,1610342842
549,1218,Arlit,18.7369,7.3853,68.18,23,0,7.76,NE,1610342932
580,1263,Kintampo,8.0563,-1.7306,72.75,80,0,2.91,GH,1610342958


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
ideal_df.count()

Unnamed: 0    7
City          7
Lat           7
Lng           7
Max Temp      7
Humidity      7
Cloudiness    7
Wind Speed    7
Country       7
Date          7
dtype: int64

In [7]:
hotel_df = ideal_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,61,San Rafael,-34.6177,-68.3301,69.55,65,0,2.46,AR,1610342232,
167,269,Gisborne,-38.6533,178.0042,72.00,73,0,4.00,NZ,1610342378,
339,718,Paraíso,9.8383,-83.8656,69.80,73,0,6.91,CR,1610342640,
429,968,Wamba,8.9333,8.6000,68.11,25,0,1.90,NG,1610342785,
459,1064,Puntarenas,9.9763,-84.8384,69.80,73,0,6.91,CR,1610342842,
549,1218,Arlit,18.7369,7.3853,68.18,23,0,7.76,NE,1610342932,
580,1263,Kintampo,8.0563,-1.7306,72.75,80,0,2.91,GH,1610342958,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng from hotel_df to find nearby hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and iterate through lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = None


In [9]:
hotel_df = hotel_df.dropna(how="any")
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,61,San Rafael,-34.6177,-68.3301,69.55,65,0,2.46,AR,1610342232,Suter Petit Hotel
167,269,Gisborne,-38.6533,178.0042,72.00,73,0,4.00,NZ,1610342378,Portside Hotel - Heritage Collection
339,718,Paraíso,9.8383,-83.8656,69.80,73,0,6.91,CR,1610342640,Hotel Rio Perlas Spa and Resort
429,968,Wamba,8.9333,8.6000,68.11,25,0,1.90,NG,1610342785,Roma Zee Guest House Annex
459,1064,Puntarenas,9.9763,-84.8384,69.80,73,0,6.91,CR,1610342842,Hotel Las Brisas
549,1218,Arlit,18.7369,7.3853,68.18,23,0,7.76,NE,1610342932,Hôtel Telwa Bungalow
580,1263,Kintampo,8.0563,-1.7306,72.75,80,0,2.91,GH,1610342958,Hamasei Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))